In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [3]:
train_dataset = torchvision.datasets.MNIST(root = './data', train = True, download = True,transform = transform)
test_dataset = torchvision.datasets.MNIST(root = './data', train = False, download = True,transform = transform)

In [4]:
train_loader = DataLoader(dataset = train_dataset,batch_size=64,shuffle = True)
test_loader = DataLoader(dataset = test_dataset,batch_size=64,shuffle = True)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
device = torch.device("cpu")

In [7]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.01)

In [8]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


Epoch [1/5], Step [100/938], Loss: 2.3118
Epoch [1/5], Step [200/938], Loss: 2.2901
Epoch [1/5], Step [300/938], Loss: 2.2932
Epoch [1/5], Step [400/938], Loss: 2.2868
Epoch [1/5], Step [500/938], Loss: 2.2848
Epoch [1/5], Step [600/938], Loss: 2.2876
Epoch [1/5], Step [700/938], Loss: 2.3034
Epoch [1/5], Step [800/938], Loss: 2.3042
Epoch [1/5], Step [900/938], Loss: 2.3074
Epoch [1/5], Loss: 2.3091
Epoch [2/5], Step [100/938], Loss: 2.3089
Epoch [2/5], Step [200/938], Loss: 2.2969
Epoch [2/5], Step [300/938], Loss: 2.3131
Epoch [2/5], Step [400/938], Loss: 2.3092
Epoch [2/5], Step [500/938], Loss: 2.3034
Epoch [2/5], Step [600/938], Loss: 2.3029
Epoch [2/5], Step [700/938], Loss: 2.2922
Epoch [2/5], Step [800/938], Loss: 2.3139
Epoch [2/5], Step [900/938], Loss: 2.3045
Epoch [2/5], Loss: 2.3023
Epoch [3/5], Step [100/938], Loss: 2.2878
Epoch [3/5], Step [200/938], Loss: 2.3102
Epoch [3/5], Step [300/938], Loss: 2.2996
Epoch [3/5], Step [400/938], Loss: 2.3125
Epoch [3/5], Step [500/9

In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images,labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'accuracy of the model in 10K test images: { 100*correct/total:.2f}%')
        

accuracy of the model in 10K test images: 11.35%


In [10]:
print(model.state_dict())


OrderedDict([('conv1.weight', tensor([[[[ 0.1790,  0.0171, -0.0492],
          [ 0.0183, -0.2940, -0.1788],
          [-0.1176, -0.0595,  0.1987]]],


        [[[ 0.0819, -0.1795, -0.2144],
          [-0.0476, -0.1579, -0.1812],
          [ 0.1156, -0.0925,  0.3217]]],


        [[[-0.1700, -0.3278,  0.2338],
          [-0.0353, -0.1386,  0.1100],
          [-0.3097, -0.1877, -0.0698]]],


        [[[ 0.1004,  0.0475, -0.1112],
          [-0.0357,  0.1920, -0.1778],
          [-0.0320, -0.2254,  0.0225]]],


        [[[-0.1152,  0.1768,  0.2580],
          [-0.2440,  0.1500, -0.2031],
          [-0.2912, -0.1200, -0.2583]]],


        [[[-0.2218,  0.0642, -0.2560],
          [-0.1577, -0.1655,  0.1115],
          [ 0.2533, -0.0333,  0.0366]]],


        [[[-0.2483,  0.2072,  0.1879],
          [-0.1223, -0.2657, -0.2189],
          [-0.2775,  0.1041,  0.2342]]],


        [[[ 0.0928, -0.3198, -0.1117],
          [ 0.0782,  0.2432, -0.0880],
          [ 0.1022, -0.2952, -0.0337]]],


  

In [11]:
epochs = 10
for epoch in range(epochs):
    model.train()
    runnin_loss = 0
    for i,(images,labels) in enumerate(train_loader):
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs,labels)

        loss.backward()
        optimizer.step()

        runnin_loss += loss.item()

        if(i+1)%100 == 0:
            print({(epoch + 1)/(epochs)})
            
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

{0.1}
{0.1}
{0.1}
{0.1}
{0.1}
{0.1}
{0.1}
{0.1}
{0.1}
Epoch [1/5], Loss: 2.3021
{0.2}
{0.2}
{0.2}
{0.2}
{0.2}
{0.2}
{0.2}
{0.2}
{0.2}
Epoch [2/5], Loss: 2.3021
{0.3}
{0.3}
{0.3}
{0.3}
{0.3}
{0.3}
{0.3}
{0.3}
{0.3}
Epoch [3/5], Loss: 2.3021
{0.4}
{0.4}
{0.4}
{0.4}
{0.4}
{0.4}
{0.4}
{0.4}
{0.4}
Epoch [4/5], Loss: 2.3021
{0.5}
{0.5}
{0.5}
{0.5}
{0.5}
{0.5}
{0.5}
{0.5}
{0.5}
Epoch [5/5], Loss: 2.3021
{0.6}
{0.6}
{0.6}
{0.6}
{0.6}
{0.6}
{0.6}
{0.6}
{0.6}
Epoch [6/5], Loss: 2.3021
{0.7}
{0.7}
{0.7}
{0.7}
{0.7}
{0.7}
{0.7}
{0.7}
{0.7}
Epoch [7/5], Loss: 2.3021
{0.8}
{0.8}
{0.8}
{0.8}
{0.8}
{0.8}
{0.8}
{0.8}
{0.8}
Epoch [8/5], Loss: 2.3021
{0.9}
{0.9}
{0.9}
{0.9}
{0.9}
{0.9}
{0.9}
{0.9}
{0.9}
Epoch [9/5], Loss: 2.3021
{1.0}
{1.0}
{1.0}
{1.0}
{1.0}
{1.0}
{1.0}
{1.0}
{1.0}
Epoch [10/5], Loss: 2.3021
